In [1]:
import json
import geopandas as gpd

In [2]:
import data.sentinel_cog as sc


In [3]:
#data from http://geojson.io 
data={
  "type": "FeatureCollection",
  "features": [
    {
      "type": "Feature",
      "properties": {},
      "geometry": {
        "type": "Polygon",
        "coordinates": [
          [
            [
              38.54827880859375,
              36.90378362619561
            ],
            [
              40.0067138671875,
              36.90378362619561
            ],
            [
              40.0067138671875,
              37.89219554724437
            ],
            [
              38.54827880859375,
              37.89219554724437
            ],
            [
              38.54827880859375,
              36.90378362619561
            ]
          ]
        ]
      }
    }
  ]
}

In [14]:
harran={
"type": "FeatureCollection",
"name": "Harran_AOI",
"crs": { "type": "name", "properties": { "name": "urn:ogc:def:crs:OGC:1.3:CRS84" } },
"features": [
{ "type": "Feature", "properties": { "fid": 1 }, "geometry": { "type": "Polygon", "coordinates": [ [ [ 38.680367610550277, 36.71081864055018 ], [ 38.688923838082594, 37.274751818816462 ], [ 39.282414893278698, 37.277085335416182 ], [ 39.29019328194444, 36.690594830019251 ], [ 38.680367610550277, 36.71081864055018 ] ] ] } }
]
}


In [17]:
datajson=json.dumps(harran)
target_area=gpd.read_file(datajson)


In [18]:
tiles_intersection,tile_map=sc.find_sentinel_tile(target_area,sentinel_tiles_path='../data/raw/boundries/sentinel_tr_tiles.shp')
tile_map

In [20]:
tiles_intersection

,name,folders,descriptio,altitude,alt_mode,time_begin,time_end,time_when,GID_0,NAME_0,geometry
0,37SDA,Features,TILE PROPERTIES<br><table border=0 cellpadding...,0.0,None,None,None,None,TUR,Turkey,"POLYGON Z ((37.87506 37.04125 0.00000, 39.1097..."
1,37SDB,Features,TILE PROPERTIES<br><table border=0 cellpadding...,0.0,None,None,None,None,TUR,Turkey,"POLYGON Z ((37.86147 37.94208 0.00000, 39.1110..."
2,37SEA,Features,TILE PROPERTIES<br><table border=0 cellpadding...,0.0,None,None,None,None,TUR,Turkey,"POLYGON Z ((38.99978 37.04658 0.00000, 40.2344..."
3,37SEB,Features,TILE PROPERTIES<br><table border=0 cellpadding...,0.0,None,None,None,None,TUR,Turkey,"POLYGON Z ((38.99977 37.94759 0.00000, 40.2493..."


In [8]:
boundry=list(target_area.geometry.bounds.values[0]) #boundry from your AOI
bbox=[boundry[0],boundry[1],boundry[2],boundry[3]] #(min lon, min lat, max lon, max lat)
dates = '2018-07-01/2018-07-12'
band_list=['B02','B03','B04','B08']
cloud_percentage=20

In [9]:
stac_result=sc.find_stac_result(bbox,dates,cloud_percentage)

In [10]:
tile_list=sc.create_tiles_list(stac_result)
tile_list

['37SDA', '37SDB', '37SDC', '37SEA', '37SEB', '37SEC']

In [ ]:
#drop_list=['37SDA', '37SDC', '37SEA', '37SEB', '37SEC']

In [ ]:
#tiles_intersection=sc.drop_tile(tiles_intersection,drop_list)
#tiles_intersection

In [54]:
from dask.distributed import Client, LocalCluster
import multiprocessing as mp
import xarray as xr
import matplotlib.pyplot as plt
import rioxarray
import satsearch

     
def find_stac_result(target_aoi,date,max_cloud=10):
    URL='https://earth-search.aws.element84.com/v0'
    with LocalCluster(n_workers=int(0.5 * mp.cpu_count()),
                          processes=True,
                          threads_per_worker=2,
                          memory_limit='2GB',
                          #ip='tcp://localhost:9895',
                          ) as cluster, Client(cluster) as client:
        results = satsearch.Search.search(url=URL,
                                          collections=['sentinel-s2-l2a-cogs'], # note collection='sentinel-s2-l2a-cogs' doesn't work
                                          datetime=date,
                                          bbox=target_aoi,
                                          query={'eo:cloud_cover': {'lt':max_cloud}}, )
    return results

def create_tiles_list(stac_result):
    items = stac_result.items()
    items_json=items.geojson()
    items_json=json.dumps(items_json)
    df=gpd.read_file(items_json)
    df['tile']=df.apply(lambda row: str(row['sentinel:utm_zone'])+row['sentinel:latitude_band']+row['sentinel:grid_square'], axis=1)
    tiles_list = sorted(df['tile'].unique().tolist())
    return tiles_list
    

In [55]:
def find_sentinel_item(stac_result,tiles_list=[],min_coverage=95,max_cloud=2):
    if tiles_list:
        tile_result_list=[]
        with LocalCluster(n_workers=int(0.5 * mp.cpu_count()),
                          processes=True,
                          threads_per_worker=1,
                          memory_limit='2GB',
                          #ip='tcp://localhost:9895',
                          ) as cluster, Client(cluster) as client:
            
            for tile in tiles_list:
                #find best image for target tile 
                tile_number=int(tile[0:2])
                lat_band=tile[2]
                grid_sq=tile[3:]
                items = stac_result.items()
                items.filter('sentinel:utm_zone',[tile_number,])
                items.filter('sentinel:latitude_band',[lat_band])
                items.filter('sentinel:grid_square', [grid_sq])
                try:
                    coverage=sorted(items.properties('sentinel:data_coverage'))
                    coverage=[c for c in coverage if c>min_coverage]
                    items.filter('sentinel:data_coverage',coverage)
                    if 100 in items.properties('sentinel:data_coverage'):
                        #get the index of value that data_coverage==100
                        dc_index=[i for i, x in enumerate(items) if items[i].properties['sentinel:data_coverage']==100]
                        #get images cloud info
                        filtered_list=[items[x].properties['eo:cloud_cover'] for i, x in enumerate(dc_index) if items[x].properties['eo:cloud_cover']<max_cloud]
                        #get first cloud cover after sorting
                        try:
                            # if data exist we use filter method
                            selected_item=sorted(filtered_list)[0]
                            items.filter('sentinel:data_coverage',[100])
                        except:
                            selected_item=sorted(items.properties('eo:cloud_cover'))[0]
            
                        items.filter('eo:cloud_cover', [selected_item])
                        tile_result_list.append(items[0])
    
                    else:
                        selected_item=sorted(items.properties('eo:cloud_cover'))[0]
                        #select best image
                        items.filter('eo:cloud_cover', [selected_item])
                        #get newest image
                        tile_result_list.append(items[0])
                              
                except:
                    # threshold ekle -2 yerine
                    coverage=sorted(items.properties('sentinel:data_coverage'))[-2:]
                    items.filter('sentinel:data_coverage',coverage)
                    selected_item=sorted(items.properties('eo:cloud_cover'))[0]
                    #select best image
                    items.filter('eo:cloud_cover', [selected_item])
                    #get newest image
                    tile_result_list.append(items[0])
            
            return tile_result_list
    
    else:
        # code define best image for each tile from stac result
        # function return list of tile's information
        tiles_list=create_tiles_list(stac_result)
        #we collect each tile's result
        tile_result_list=[]
        with LocalCluster(n_workers=int(0.5 * mp.cpu_count()),
                          processes=True,
                          threads_per_worker=1,
                          memory_limit='2GB',
                          #ip='tcp://localhost:9895',
                          ) as cluster, Client(cluster) as client:
            for tile in tiles_list:
                tile_number=int(tile[0:2])
                lat_band=tile[2]
                grid_sq=tile[3:]
                items = stac_result.items()
                items.filter('sentinel:utm_zone',[tile_number,])
                items.filter('sentinel:latitude_band',[lat_band])
                items.filter('sentinel:grid_square', [grid_sq])
                try:
                    coverage=sorted(items.properties('sentinel:data_coverage'))
                    coverage=[c for c in coverage if c>min_coverage]
                    items.filter('sentinel:data_coverage',coverage)
                    if 100 in items.properties('sentinel:data_coverage'):
                        #get the index of value that data_coverage==100
                        dc_index=[i for i, x in enumerate(items) if items[i].properties['sentinel:data_coverage']==100]
                        #get images cloud info
                        filtered_list=[items[x].properties['eo:cloud_cover'] for i, x in enumerate(dc_index) if items[x].properties['eo:cloud_cover']<max_cloud]
                        #get first cloud cover after sorting
                        try:
                            # if data exist we use filter method
                            selected_item=sorted(filtered_list)[0]
                            items.filter('sentinel:data_coverage',[100])
    
                        except:
                            selected_item=sorted(items.properties('eo:cloud_cover'))[0]
            
                        items.filter('eo:cloud_cover', [selected_item])
                        tile_result_list.append(items[0])
                        
                    else:
                        selected_item=sorted(items.properties('eo:cloud_cover'))[0]
                        #select best image
                        items.filter('eo:cloud_cover', [selected_item])
                        #get newest image
                        tile_result_list.append(items[0])
                              
                except:
                    coverage=sorted(items.properties('sentinel:data_coverage'))[-2:]
                    items.filter('sentinel:data_coverage',coverage)
                    selected_item=sorted(items.properties('eo:cloud_cover'))[0]
                    #select best image
                    items.filter('eo:cloud_cover', [selected_item])
                    #get newest image
                    tile_result_list.append(items[0])
            
            return tile_result_list

In [11]:
%%time
stac_result=sc.find_stac_result(bbox,dates,cloud_percentage)
items_list=sc.find_sentinel_item(stac_result=stac_result)
#Wall time: 37.3 s

Wall time: 6.05 s


In [12]:
items_list

[S2A_37SDA_20180712_0_L2A,
 S2A_37SDB_20180712_0_L2A,
 S2B_37SDC_20180707_0_L2A,
 S2A_37SEA_20180712_0_L2A,
 S2A_37SEB_20180702_0_L2A,
 S2B_37SEC_20180707_0_L2A]

In [67]:
len(a)

13

In [ ]:
%%time
from dask.distributed import Client, LocalCluster
import multiprocessing as mp
import xarray as xr
import matplotlib.pyplot as plt
import rioxarray
with LocalCluster(n_workers=int(0.5 * mp.cpu_count()),
                          processes=False,
                          threads_per_worker=2,
                          memory_limit='2GB',
                          #ip='tcp://localhost:9895',
                          ) as cluster, Client(cluster) as client:
    stac_result=sc.find_stac_result(bbox,dates,cloud_percentage)
    items_list=sc.find_sentinel_item(stac_result=stac_result)
#Wall time: 42.8 s

In [ ]:
%%time
from dask.distributed import Client, LocalCluster
import multiprocessing as mp
import xarray as xr
import matplotlib.pyplot as plt
import rioxarray
with LocalCluster(n_workers=int(0.5 * mp.cpu_count()),
                          processes=True,
                          threads_per_worker=1,
                          memory_limit='2GB',
                          #ip='tcp://localhost:9895',
                          ) as cluster, Client(cluster) as client:
    stac_result=sc.find_stac_result(bbox,dates,cloud_percentage)
    items_list=sc.find_sentinel_item(stac_result=stac_result)
#Wall time: 48 s


In [ ]:
%%time
from dask.distributed import Client, LocalCluster
import multiprocessing as mp
import xarray as xr
import matplotlib.pyplot as plt
import rioxarray
with LocalCluster(n_workers=int(0.5 * mp.cpu_count()),
                          processes=True,
                          threads_per_worker=1,
                          memory_limit='2GB',
                          #ip='tcp://localhost:9895',
                          ) as cluster, Client(cluster) as client:
    stac_result=sc.find_stac_result(bbox,dates,cloud_percentage)
    items_list=sc.find_sentinel_item(stac_result=stac_result)


In [ ]:
sc.show_result_map(items_list=items_list,overview=False,target_area=target_area)

In [ ]:
sc.show_result_map(items_list=items_list,overview=True,target_area=target_area)

In [ ]:
sc.show_items_list(items_list[0:1])

In [ ]:
df=sc.show_result_df(result=stac_result)

In [ ]:
df.head()

In [ ]:
df[df['sentinel:grid_square']=='DC']

In [ ]:
df[df['sentinel:grid_square']=='DB'].iloc[1]['sentinel:product_id']

In [ ]:
items_list

In [ ]:
sc.download_image(item_list=items_list,band_list=['thumbnail'])

In [ ]:
%%time
subset=sc.download_subset_image(download_status=False,item_list=items_list,aoi=data,target_epsg='',
                        band_list=['B02','B08'],download_path='./sentinel_cog',name_suffix='',auto_folder=True)

In [ ]:
img1=subset[0]
img1

In [ ]:
from dask.distributed import Client, LocalCluster
import multiprocessing as mp
import xarray as xr
import matplotlib.pyplot as plt
import rioxarray
band_list=['B01', 'B02', 'B03', 'B04', 'B05', 'B06', 'B07', 'B08', 'B8A',
           'B09', 'B11', 'B12', 'AOT', 'WVP', 'SCL', 'info', 'metadata',
           'visual', 'overview', 'thumbnail']

def download_subset_image(download_status=False,stac_result=None,item_id_list=[],
                        item_list=[],aoi=None,target_epsg='',
                        band_list=band_list[:-5],
                        download_path='./sentinel_cog',name_suffix='',auto_folder=True):
    result_list=[]
    if item_list:
        with LocalCluster(n_workers=int(0.5 * mp.cpu_count()),
                          processes=False,
                          threads_per_worker=1,
                          memory_limit='2GB',
                          #ip='tcp://localhost:9895',
                          ) as cluster, Client(cluster) as client:
            for item in item_list:
                bands_dict={}
                for band in band_list:
                    img_name=item.properties['sentinel:product_id']
                    bands_dict['image_name']=img_name
                    band_url=item.assets[band]['href']
                    rds = rioxarray.open_rasterio(band_url, masked=True, chunks=(512,512))
                    #aoi data from http://geojson.io 
                    # get aoi as geopandas df
                    datajson=json.dumps(aoi)
                    target_area=gpd.read_file(datajson)
                    #https://geopandas.org/projections.html
                    target_area=target_area.to_crs(rds.rio.crs.to_string())
                    clipped =rds.rio.clip(target_area.geometry)
                     
                    if target_epsg:
                        # target_epsg='epsg:4326'
                        clipped = clipped.rio.reproject(target_epsg)
    
                    if download_status:
                        img_path=download_path+'/'+img_name
                        if not os.path.isdir(img_path):
                            os.mkdir(img_path)
                        img_name=band+'.tif'
                        clipped.rio.to_raster(img_path+'/'+img_name)
                       
                    bands_dict[band]=clipped.copy()
                    rds=None
                result_list.append(bands_dict)
                
        return result_list

In [ ]:
%%time
subset_dask=download_subset_image(download_status=False,item_list=items_list,aoi=data,target_epsg='',
                        band_list=['B02','B08'],download_path='./sentinel_cog',name_suffix='',auto_folder=True)

In [13]:
import dask_try

ModuleNotFoundError: No module named 'dask_try'

In [ ]:
%%time
ndviwitharray=dask_try.calculate_ndvi(subset_dask[0]['B02'],subset_dask[0]['B08'])

In [ ]:
type(ndviwitharray)

In [ ]:
%%time
ndvidirecty=dask_try.direct_ndvi(subset_dask[0]['B02'],subset_dask[0]['B08'])

In [ ]:
from dask import 

In [ ]:
red_xarray=None
nir_xarray=None
red=None
nir=None
red=None
nir=None
ndvi = None

In [ ]:
%%time
with LocalCluster(n_workers=int(0.6 * mp.cpu_count()),
    processes=True,
    threads_per_worker=1,
    memory_limit='2GB',
    #ip='tcp://localhost:9895',
) as cluster, Client(cluster) as client:
    red_xarray=subset_dask[0]['B02']
    nir_xarray=subset_dask[0]['B08']
    red=red_xarray.persist()
    nir=nir_xarray.persist()
    red=red.values
    nir=nir.values
    ndvi = (nir.astype(float) - red.astype(float))/(nir + red)
    
    #2min 16s

In [ ]:
%%time
with LocalCluster(n_workers=int(0.6 * mp.cpu_count()),
    processes=False,
    threads_per_worker=1,
    memory_limit='2GB',
    #ip='tcp://localhost:9895',
) as cluster, Client(cluster) as client:
    red_xarray=subset_dask[0]['B02']
    nir_xarray=subset_dask[0]['B08']
    red=red_xarray.persist()
    nir=nir_xarray.persist()
    red=red.values
    nir=nir.values
    ndvi = (nir.astype(float) - red.astype(float))/(nir + red)
#2.31 s
    

In [ ]:
%%time
with LocalCluster(n_workers=int(0.6 * mp.cpu_count()),
    processes=True,
    threads_per_worker=1,
    memory_limit='2GB',
    #ip='tcp://localhost:9895',
) as cluster, Client(cluster) as client:
    red_xarray=subset_dask[0]['B02']
    nir_xarray=subset_dask[0]['B08']
    ndvi = (nir_xarray.astype(float) - red_xarray.astype(float))/(nir_xarray + red_xarray)
    ndvi.rio.to_raster('sen_cog_ndvi.tif')

In [ ]:
with LocalCluster(n_workers=int(0.6 * mp.cpu_count()),
    processes=True,
    threads_per_worker=1,
    memory_limit='2GB',
    #ip='tcp://localhost:9895',
) as cluster, Client(cluster) as client:
    ndvi_reproject = ndvi.rio.reproject_match(red_xarray)

In [ ]:
ndvi_reproject.rio.crs

In [ ]:
ndvi_4326 = ndvi.rio.reproject('epsg:4326')

In [ ]:
%%time
with LocalCluster(n_workers=int(0.6 * mp.cpu_count()),
    processes=True,
    threads_per_worker=1,
    memory_limit='2GB',
    #ip='tcp://localhost:9895',
) as cluster, Client(cluster) as client:
    con_utm=ndvi_4326.rio.reproject(dst_crs=ndvi.rio.crs,shape=ndvi.rio.shape)
    con_utm=con_utm.rio.reproject_match(ndvi)
    con_utm.rio.to_raster('ndvilatToUTM.tif')

In [ ]:
%%time
with LocalCluster(n_workers=int(0.6 * mp.cpu_count()),
    processes=False,
    threads_per_worker=1,
    memory_limit='2GB',
    #ip='tcp://localhost:9895',
) as cluster, Client(cluster) as client:
    con_utm=ndvi_4326.rio.reproject(dst_crs=ndvi.rio.crs,shape=ndvi.rio.shape)
    con_utm=con_utm.rio.reproject_match(ndvi)
    con_utm.rio.to_raster('ndvilatToUTM2.tif')

In [ ]:
ndvi_4326.rio.reproject?

In [ ]:
ndvi.rio.reproject_match?

In [ ]:
def to_raster(in_xds, template_xds, out_file):
    in_xds = in_xds.rio.write_crs(template_xds.rio.crs)
    if template_xds.rio.nodata is not None:
        in_xds.attrs["_FillValue"] = template_xds.rio.nodata
    in_xds.rio.to_raster(out_file)



In [ ]:
%%time
with LocalCluster(n_workers=int(0.6 * mp.cpu_count()),
    processes=True,
    threads_per_worker=1,
    memory_limit='2GB',
    #ip='tcp://localhost:9895',
) as cluster, Client(cluster) as client:
    to_raster(ndvi_4326,red_xarray, "ndvifromred.tif")

In [ ]:
geo=new_target.__geo_interface__

In [ ]:
from rasterstats import zonal_stats, point_query

In [ ]:
stats = zonal_stats(geo,ndvi,stats=['min', 'max', 'median', 'majority', 'sum'])

In [ ]:
new_target=target_area.to_crs(ndvi.rio.crs.to_string())

In [ ]:
aoi_gpd=target_area.to_crs(ndvi.rio.crs.to_string())

In [ ]:
aoi_gpd.geometry

In [ ]:
clipped =rds.rio.clip(aoi_gpd.geometry)

In [ ]:
clipped.rio.to_raster('denemeee.tif')

In [ ]:
aoi_gpd.to_file('utm',driver='ESRI Shapefile')

In [ ]:
target_area.to_file('latlon',driver='ESRI Shapefile')

In [ ]:
res={'type': 'Polygon',
 'coordinates': [[[37.876381589019886, 36.95257399124932],
   [37.861482438916816, 37.942072015005024],
   [38.5700476360887, 37.9467949936599],
   [38.48396578520234, 37.68856019631257],
   [38.238959267471955, 36.955451379212235],
   [37.876381589019886, 36.95257399124932]]]}

In [ ]:
datajson=json.dumps(res)
target_area=gpd.read_file(datajson)
target_area.geometry

In [ ]:
target_area.to_file('latlon2',driver='ESRI Shapefile')

In [ ]:
aoi_gpd=target_area.to_crs(sdb37.rio.crs.to_string())

In [ ]:
aoi_gpd.to_file('utm',driver='ESRI Shapefile')